# Merge label & token

In [ ]:
import pandas as pd
import xlrd
import xlwt
from collections import Counter
import time

import numpy as np
import csv
pd.set_option('display.max_rows', None)

In [ ]:
# combine label and token together
dflabel = pd.read_csv('/data/label_test_1125.txt')
dftoken = pd.read_csv('/data/token1_test.txt', quoting=csv.QUOTE_NONE,error_bad_lines=False, encoding='utf-8')


In [ ]:
dflabel.merge(dftoken, how='left', left_index=True, right_index=True).to_excel('NER_tmall_1125.xlsx')


# BIO to result

In [38]:
# import the data 
# output from NER model
output_from_bert = pd.read_excel('NER_tmall_1125.xlsx', engine = 'openpyxl')

# 业务输出需要预测的数据文件
result = pd.read_excel('new_concept_2022-10.xlsx', engine = 'openpyxl')

In [39]:
output_from_bert.columns = ['index', 'token', 'word']

In [40]:
output_from_bert.head(10)

,index,token,word
0,0,O,劲
1,1,O,度
2,2,O,n
3,3,O,9
4,4,O,5
5,5,O,型
6,6,O,口
7,7,O,罩
8,8,O,医
9,9,O,用


In [41]:
output_from_bert[['index', 'token', 'word']] = output_from_bert[['index', 'token', 'word']].astype(str)

In [42]:
output_from_bert.shape

(791835, 3)

In [43]:
def test_filter(string):
    tag = string
    split_tag = tag.split(',')
    tag_res = set(split_tag)
    s = ','
    res = s.join(tag_res)
    return res

In [49]:
# BIO处理成实体，result新增key_words和process_key_words两列
file2 = '新锐赛道头部商品11月'
# insert 2 new columns 
col_name = result.columns.tolist()
index = col_name.index('spu_name') + 1
col_name.insert(index, 'key_words')
index = col_name.index('key_words') + 1
col_name.insert(index, 'process_key_words')

result = result.reindex(columns=col_name)
tag = ''
num = 0
for index, row in output_from_bert.iterrows():
    if row[1] == 'O':
        if len(tag) > 0 and tag[-1] != ',':
            tag += ','
    if row[1] == '[CLS]':
        tag = ''
    if row[1] == '[SEP]':
        while len(tag) > 0 and tag[-1] == ',':
            tag = tag[:-1]
        if len(tag) == 0:
            tag = '1'
        # print(str(tag))
        result.loc[num, 'key_words'] = str(tag)
        count = 0         # 进一步处理：去掉只有单个字的关键词
        flag = list()
        for i in range(len(tag)):
            count += 1
            if tag[i] == ',':
                if count == 2:
                    flag.append(i)
                count = 0
        if count == 1:
            tag = tag[:-2]
        for j in range(len(flag)):
            if flag[j] - 2 * j - 2 < 0:
                tag = tag[2:]
            else:
                tag = tag[:flag[j] - 2 * j - 2] + tag[flag[j] - 2 * j:]
        tag = test_filter(tag)
        result.loc[num, 'process_key_words'] = str(tag)
        num += 1
        tag = ''
    if row[1] == 'B':
        if len(tag) > 0 and tag[-1] != ',':
            tag += ','
        tag += row[2]
    if row[1] == 'I':
        tag += row[2]
    if row[1] == 'E':
        tag += row[2]
result.to_excel(file2 + '-result.xls')

/var/folders/7y/cwfy92wj6rq6_kc_6n81nb1m0000gp/T/ipykernel_15882/2323161656.py:53: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  result.to_excel(file2 + '-result.xls')


In [45]:
# 返回一个list，包含"规则表"里已有的keywords和"百观标准品类"里c1,c2,c3的内容
rule_table = pd.read_excel('/tools/EC-新锐赛道规则表-规则表.xlsx', engine='openpyxl')
category_table = pd.read_excel('/tools/百观标准品类.xlsx', engine='openpyxl')
include = list()
for data in rule_table['spu_include'].values:
    data = data.strip('"')
    split_tag = data.split(',')
    include += split_tag
include = list(set(include))
include = include + list(set(category_table['bg1_c1'].values)) + list(set(category_table['bg1_c2'].values)) + list(set(category_table['bg1_c3'].values))
print(len(include))

1778


In [46]:
# 得到废词
all_data = pd.read_excel('/tools/关键词all.xlsx', engine='openpyxl')
included_keywords = include
all_keywords = list()
for data in all_data['新概念词'].values:
    data = str(data)
    # data = data.strip(' ')
    split_tag = data.split('、')
    all_keywords += split_tag
all_keywords = list(set(all_keywords))
obsolete = [val for val in all_keywords if val not in included_keywords]
print(len(all_keywords),len(included_keywords),len(obsolete))

17684 1778 16959


In [50]:
#### 把有多个关键词拆成多列
# read the data 
start = time.time()
workbook = xlrd.open_workbook('新锐赛道头部商品11月-result.xls')
booksheet = workbook.sheet_by_name('Sheet1')

write_workbook = xlwt.Workbook(encoding='utf-8')
booksheet02 = write_workbook.add_sheet('Sheet 1', cell_overwrite_ok=True)
col_ = 0
col = 0
for row in range(booksheet.nrows):
    row_content = booksheet.row_values(row)
    col_5 = row_content[5].split(',')
    col_ += len(col_5)
    for j in range(0, len(col_5)):
        rowdata = [row_content[0], row_content[1], row_content[2], row_content[3], row_content[4], 
                   col_5[j]]
        if col < col_:
            for i in range(len(rowdata)):
                booksheet02.write(col, i, rowdata[i])
            col += 1
end = time.time()
duration = end - start
print("Duration for this process is " + str(duration/60) + ' seconds')
write_workbook.save('新锐赛道头部商品11月-result_v1.xls')
#write_workbook.close()

Duration for this process is 0.021295050779978435 seconds


In [51]:
table = pd.read_excel('新锐赛道头部商品11月-result_v1.xls')
table.head(10)

,Unnamed: 0,spu_id,platform,spu_name,key_words,process_key_words
0,0,100032848215,1,劲度 N95型口罩医用一次性防护口罩 n95级医用双层熔喷布防细菌飞沫雾霾透气成人男女通用1...,1,NaN
1,1,100039268128,1,振德（ZHENDE）一次性成人医用外科国庆口罩中国庆包装款玫红渐变色独立包装10只/盒（二类...,1,NaN
2,2,100027702188,1,雅培（Abbott）血糖仪家用瞬感动态监测血糖仪免采血探头传感器 1个装 【大促超值13赠1...,"血糖仪,血糖仪",血糖仪
3,3,908664,1,欧姆龙（OMRON）电子血压计家用上臂式大画面 医用测量血压仪 U724J 医用级全自动臂筒...,电子血压计,电子血压计
4,4,100005926666,1,三诺血糖仪家用 语音播报智能医用 脱氢酶血糖试纸 GA-3型套装（仪器+50支试纸+50支采...,1,NaN
5,5,100039268318,1,可孚 新冠抗原检测盒病毒快速自测核酸试剂盒新型冠状家庭家用公司用试纸包【现货速发20人份】 ...,"抗原检测盒,核酸试剂盒",抗原检测盒
6,5,100039268318,1,可孚 新冠抗原检测盒病毒快速自测核酸试剂盒新型冠状家庭家用公司用试纸包【现货速发20人份】 ...,"抗原检测盒,核酸试剂盒",核酸试剂盒
7,6,522676,1,小林制药 小林暖宝宝 暖贴暖宝宝贴膝盖关节膝肘兼用暖身贴保暖自发热贴6片或6+赠2片（两款随...,1,NaN
8,7,100011271419,1,鱼跃(YUWELL)雾化器雾化机儿童成人家用医用压缩雾化器403M 空气压缩式雾化仪面罩 店...,"雾化器,雾化仪,雾化器",雾化仪
9,7,100011271419,1,鱼跃(YUWELL)雾化器雾化机儿童成人家用医用压缩雾化器403M 空气压缩式雾化仪面罩 店...,"雾化器,雾化仪,雾化器",雾化器


## Word frequency

In [52]:
# 统计词频率
table = pd.read_excel('新锐赛道头部商品11月-result_v1.xls')
keywords = list(table['process_key_words'])
counter = Counter(keywords)
dictionary = dict(counter)

col_name = table.columns.tolist()
index = col_name.index('process_key_words') + 1
col_name.insert(index, 'frequency')

table = table.reindex(columns=col_name)

for index, row in table.iterrows():
    x = str(row[5])
    if x != "nan":
        row[6] = dictionary[row[5]]
    else:
        row[6] = row[5]
    table.loc[index, 'frequency'] = str(row[6])
table.to_excel('新锐赛道头部商品11月-result_v2.xlsx')

## Produce new word

In [53]:
#  新增词保存到newwords.txt
spu_include_data = include

table = pd.read_excel('新锐赛道头部商品11月-result_v2.xlsx')
col_name = table.columns.tolist()
index = col_name.index('frequency') + 1
col_name.insert(index, 'new_keyword')
table = table.reindex(columns=col_name)
new_word=list()
for index, row in table.iterrows():
    x = str(row[6])
    if x != "nan" and x not in spu_include_data:
        new_word.append(x)
        row[7] = 1
        table.loc[index, 'new_keyword'] = str(row[7])
table.to_excel('新锐赛道头部商品11月-result_v3.xlsx')
#     df = pd.read_excel("2022年1月天猫新品商品标题-split.xlsx",engine='openpyxl', usecols=[2,3,4,5,6,7,8],names=None)
#     df.to_excel('2022年1月天猫新品商品标题-split.xlsx')


## Add other columns 

In [54]:
data = pd.read_excel('新锐赛道头部商品11月-result.xls')
other_columns = data[['spu_id', 'bg1_c1', 'bg1_c2', 'bg1_c3', 'catid_path', 'gmv', 'unit_sold', 'us_mom', 'gmv_mom', 'rn']]

df = pd.read_excel('新锐赛道头部商品11月-result_v3.xlsx', engine = 'openpyxl')
full_data = df.merge(other_columns, on = ['spu_id'], how = 'left')

full_data.to_excel('新锐赛道头部商品11月-result_v3.xlsx')


In [ ]:
new_word=list(set(new_word))
f = open("newwords.txt",
         'a')
for i in range(len(new_word)):
    f.write(new_word[i])  # 将category写入文件中
    f.write("\n")  # 换行